[View in Colaboratory](https://colab.research.google.com/github/rdenadai/TxtP-Study-Notebooks/blob/master/notebooks/text_classification_example.ipynb)

## Análise e Validação de Textos em Português


### Referências:

 - Bibliotecas:
  - [NLTK](http://www.nltk.org/howto/portuguese_en.html)
  - [spaCy](https://spacy.io/usage/spacy-101)
 
 - Dados:
  - [Frases para Face](https://www.frasesparaface.com.br/outras-frases/)
 
 - Tutoriais
  - [Utilizando processamento de linguagem natural para criar uma sumarização automática de textos](https://medium.com/@viniljf/utilizando-processamento-de-linguagem-natural-para-criar-um-sumariza%C3%A7%C3%A3o-autom%C3%A1tica-de-textos-775cb428c84e)
  - [Latent Semantic Analysis (LSA) for Text Classification Tutorial](http://mccormickml.com/2016/03/25/lsa-for-text-classification-tutorial/)
  - [Machine Learning :: Cosine Similarity for Vector Space Models (Part III)](http://blog.christianperone.com/2013/09/machine-learning-cosine-similarity-for-vector-space-models-part-iii/)
  - [My Notes for Singular Value Decomposition with Interactive Code ](https://towardsdatascience.com/my-notes-for-singular-value-decomposition-with-interactive-code-feat-peter-mills-7584f4f2930a)
  - [https://plot.ly/ipython-notebooks/principal-component-analysis/](https://plot.ly/ipython-notebooks/principal-component-analysis/)
 
 - Topic Modelling
  - [Topic Modeling with LSA, PLSA, LDA & lda2Vec](https://medium.com/nanonets/topic-modeling-with-lsa-psla-lda-and-lda2vec-555ff65b0b05)
  - [Integrating Topics and Syntax (HHM-LDA)](http://psiexp.ss.uci.edu/research/papers/composite.pdf)
 
 - Others
  - [PANAS-t: A Pychometric Scale for Measuring Sentiments on Twitter](https://arxiv.org/abs/1308.1857)
  - [Um Método de Identificação de Emoções em Textos Curtos para o Português do Brasil](http://www.ppgia.pucpr.br/~paraiso/Projects/Emocoes/Emocoes.html)
  - [An Introduction to Latent Semantic Analysis](http://lsa.colorado.edu/papers/dp1.LSAintro.pdf)
  - [Unsupervised Emotion Detection from Text using Semantic and Syntactic Relations](http://www.cse.yorku.ca/~aan/research/paper/Emo_WI10.pdf)
  - [An Efficient Method for Document Categorization Based on Word2vec and Latent Semantic Analysis](https://ieeexplore.ieee.org/document/7363382)
  - [Sentiment Classification of Documents Based on Latent Semantic Analysis](https://link.springer.com/chapter/10.1007/978-3-642-21802-6_57)
  - [Applying latent semantic analysis to classify emotions in Thai text](https://ieeexplore.ieee.org/document/5486137)
  - [Text Emotion Classification Research Based on Improved Latent Semantic Analysis Algorithm](https://www.researchgate.net/publication/266651993_Text_Emotion_Classification_Research_Based_on_Improved_Latent_Semantic_Analysis_Algorithm)



### Instalação

In [1]:
!pip install -U spacy
!python -m spacy download en
!python -m spacy download pt
# !pip install feedparser

Requirement already up-to-date: spacy in /usr/local/lib/python3.6/dist-packages (2.0.12)
    100% |████████████████████████████████| 37.4MB 32.1MB/s 
  Running setup.py install for en-core-web-sm ... - \ | done

    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en

    You can now load the model via spacy.load('en')

    100% |████████████████████████████████| 38.7MB 56.7MB/s 
  Running setup.py install for pt-core-news-sm ... - \ | done

    Linking successful
    /usr/local/lib/python3.6/dist-packages/pt_core_news_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/pt

    You can now load the model via spacy.load('pt')



In [2]:
# Download Oplexicon
!rm -rf wget-log*
!rm -rf oplexicon_v3.0
!wget -O oplexicon_v3.0.zip https://github.com/rdenadai/sentiment-analysis-2018-president-election/blob/master/dataset/oplexicon_v3.0.zip?raw=true
!unzip oplexicon_v3.0.zip
!ls -lh


Redirecting output to ‘wget-log’.
Archive:  oplexicon_v3.0.zip
  inflating: oplexicon_v3.0/lexico_v3.0.txt  
  inflating: oplexicon_v3.0/README   
total 120K
drwxr-xr-x 2 root root 4.0K Oct  9 22:46 oplexicon_v3.0
-rw-r--r-- 1 root root 102K Oct  9 22:46 oplexicon_v3.0.zip
drwxr-xr-x 2 root root 4.0K Sep 28 23:32 sample_data
-rw-r--r-- 1 root root 1.6K Oct  9 22:46 wget-log


### Imports

In [3]:
import nltk

nltk.download('rslp')
nltk.download('averaged_perceptron_tagger')
nltk.download('floresta')
nltk.download('mac_morpho')
nltk.download('machado')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('words')

import concurrent.futures
import codecs
import re
import pprint
from random import shuffle
from string import punctuation
import copy
from unicodedata import normalize

import numpy as np
from scipy.sparse.linalg import svds
from scipy.linalg import svd
import pandas as pd
import spacy
from spacy.lang.pt.lemmatizer import LOOKUP

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
from sklearn.utils.extmath import randomized_svd

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import floresta as flt
from nltk.corpus import machado as mch
from nltk.corpus import mac_morpho as mcm


nlp = spacy.load('pt')
pp = pprint.PrettyPrinter(indent=4)
stemmer = nltk.stem.RSLPStemmer()

[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package floresta to /root/nltk_data...
[nltk_data]   Unzipping corpora/floresta.zip.
[nltk_data] Downloading package mac_morpho to /root/nltk_data...
[nltk_data]   Unzipping corpora/mac_morpho.zip.
[nltk_data] Downloading package machado to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


### Functions

In [0]:
def normalization(x, a, b):
    return (2 * b) * (x - np.min(x)) / np.ptp(x) + a


def remover_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')


def load_oplexicon_data(filename):
    spacy_conv = {
        'adj': 'ADJ',
        'n': 'NOUN',
        'vb': 'VERB',
        'det': 'DET',
        'emot': 'EMOT',
        'htag': 'HTAG'
    }
    
    data = {}
    with codecs.open(filename, 'r', 'UTF-8') as hf:
        lines = hf.readlines()
        for line in lines:
            info = line.lower().split(',')
            if len(info[0].split()) <= 1:
                info[1] = [spacy_conv.get(tag) for tag in info[1].split()]
                word, tags, sent = info[:3]
                if 'HTAG' not in tags and 'EMOT' not in tags:
                    word = remover_acentos(word.replace('-se', ''))
                    word = LOOKUP.get(word, word)
                    # stem = stemmer.stem(word)
                    if word in data:
                        data[word] += [{
                            'word': [word],
                            'tags': tags,
                            'sentiment': sent
                        }]
                    else:
                        data[word] = [{
                            'word': [word],
                            'tags': tags,
                            'sentiment': sent
                        }]
    return data

### Usage

In [5]:
frase = u"Gostaria de saber mais informações sobre a Amazon. Uma excelente loja de produtos online!".lower()
doc = nlp(remover_acentos(frase))
pp.pprint([(w.text, w.pos_) for w in doc])

[   ('gostaria', 'VERB'),
    ('de', 'ADP'),
    ('saber', 'VERB'),
    ('mais', 'ADV'),
    ('informacoes', 'NOUN'),
    ('sobre', 'ADP'),
    ('a', 'DET'),
    ('amazon', 'NOUN'),
    ('.', 'PUNCT'),
    ('uma', 'DET'),
    ('excelente', 'ADJ'),
    ('loja', 'NOUN'),
    ('de', 'ADP'),
    ('produtos', 'NOUN'),
    ('online', 'ADJ'),
    ('!', 'PUNCT')]


In [6]:
opx = load_oplexicon_data('oplexicon_v3.0/lexico_v3.0.txt')
print('Oplexicon size: ', len(opx))
print('Examples: ')

view = opx.items()
pp.pprint(list(view)[:7])

Oplexicon size:  15958
Examples: 
[   ('ab-rogar', [{'sentiment': '-1', 'tags': ['VERB'], 'word': ['ab-rogar']}]),
    ('ababadar', [{'sentiment': '0', 'tags': ['VERB'], 'word': ['ababadar']}]),
    (   'ababelar',
        [   {'sentiment': '-1', 'tags': ['VERB'], 'word': ['ababelar']},
            {'sentiment': '1', 'tags': ['VERB'], 'word': ['ababelar']}]),
    ('abacanar', [{'sentiment': '1', 'tags': ['VERB'], 'word': ['abacanar']}]),
    ('abacinar', [{'sentiment': '1', 'tags': ['VERB'], 'word': ['abacinar']}]),
    (   'abafar',
        [   {'sentiment': '-1', 'tags': ['ADJ'], 'word': ['abafar']},
            {'sentiment': '-1', 'tags': ['ADJ'], 'word': ['abafar']},
            {'sentiment': '-1', 'tags': ['ADJ'], 'word': ['abafar']},
            {'sentiment': '-1', 'tags': ['ADJ'], 'word': ['abafar']},
            {'sentiment': '-1', 'tags': ['VERB'], 'word': ['abafar']}]),
    (   'abafante',
        [   {'sentiment': '-1', 'tags': ['ADJ'], 'word': ['abafante']},
            {'s

In [0]:
ALEGRIA = ['abundante', 'acalmar', 'aceitável', 'aclamar', 'aconchego', 'adesão', 'admirar', 'adorar', 'afável', 'afeição', 'afeto', 'afortunado', 'agradar', 'ajeitar', 'alívio', 'amabilidade', 'amado', 'amar', 'amável', 'amenizar', 'ameno', 'amigável', 'amistoso', ' amizade', ' amor', ' animação', ' ânimo', 'anseio', 'ânsia', 'ansioso', 'apaixonado', 'apaziguar', 'aplausos', 'apoiar', 'aprazer', 'apreciar', 'aprovação', 'aproveitar', 'ardor', 'armirar', 'arrumar', 'atração', 'atraente', 'atrair', 'avidamente', 'avidez', 'ávido', 'belo', 'bem-estar', 'beneficência', 'beneficiador', 'benefício', 'benéfico', 'benevocência', 'benignamente', 'benígno', 'bom', 'bondade', 'bondoso', 'bonito', 'brilhante', 'brincadeira', 'calma', 'calor', 'caridade', 'caridoso', 'carinho', 'cativar', 'charme', 'cheery', 'clamar', 'cofortar', 'coleguismo', 'comédia', 'cômico', 'comover', 'compaixão', 'companheirismo', 'compatibilidade', 'compatível', 'complacência', 'completar', 'compreensão', 'conclusão', 'concretização', 'condescendência', 'confiança', 'confortante', 'congratulação', 'conquistar', 'consentir', 'consideração', 'consolação', 'contentamento', 'coragem', 'cordial', 'considerar', 'consolo', 'contente', 'cuidadoso', 'cumplicidade', 'dedicação', 'deleitado', 'delicadamente', 'delicadeza', 'delicado', 'desejar', 'despreocupação', 'devoção', 'devoto', 'diversão', 'divertido', 'encantar', 'elogiado', 'emoção', 'emocionante', 'emotivo', 'empatia', 'empático', 'empolgação', 'enamorar', 'encantado', 'encorajado', 'enfeitar', 'engraçado', 'entendimento', 'entusiasmadamente', 'entusiástico', 'esperança', 'esplendor', 'estima', 'estimar', 'estimulante', 'euforia', 'eufórico', 'euforizante', 'exaltar', 'excelente', 'excitar', 'expansivo', 'extasiar', 'exuberante', 'exultar', 'fã', 'facilitar', 'familiaridade', 'fascinação', 'fascínio', 'favor', 'favorecer', 'favorito', 'felicidade', 'feliz', 'festa', 'festejar', 'festivo', 'fidelidade', 'fiel', 'filantropia', 'filantrópico', 'fraterno', 'ganhar', 'generosidade', 'generoso', 'gentil', 'glória', 'glorificar', 'gostar', 'gostoso', 'gozar', 'gratificante', 'grato', 'hilariante', 'honra', 'humor', 'impressionar', 'incentivar', 'incentivo', 'inclinação', 'incrível', 'inspirar', 'interessar', 'interesse', 'irmandade', 'jovial', 'jubilante', 'júbilo', 'lealdade', 'legítimo', 'leveza', 'louvar', 'louvável', 'louvavelmente', 'lucrativo', 'lucro', 'maravilhoso', 'melhor', 'obter', 'obteve', 'ode', 'orgulho', 'paixão', 'parabenizar', 'paz', 'piedoso', 'positivo', 'prazenteiro', 'prazer', 'predileção', 'preencher', 'preferência', 'preferido', 'promissor', 'prosperidade', 'proteção', 'proteger', 'revigorar', 'simpático', 'vantajoso', 'protetor', 'risada', 'sobrevivência', 'vencedor', 'proveito', 'risonho', 'sobreviver', 'veneração', 'provilégio', 'romântico', 'sorte', 'ventura', 'querer', 'romantismo', 'sortudo', 'vida', 'radiante', 'saciar', 'sucesso', 'vigor', 'realizar', 'saciável', 'surpreender', 'virtude', 'recomendável', 'satisfação', 'tenro', 'virtuoso', 'reconhecer', 'satisfatoriamente', 'ternura', 'vitória', 'recompensa', 'satisfatório', 'torcer', 'vitorioso', 'recrear', 'satisfazer', 'tranquilo', 'viver', 'recreativo', 'satisfeito', 'tranquilo', 'vivo', 'recreação', 'sedução', 'triunfo', 'zelo', 'regozijar', 'seduzir', 'triunfal', 'zeloso', 'respeitar', 'sereno', 'triunfante', 'ressuscitar', 'simpaticamente', 'vantagem',]
DESGOSTO = ['abominável', 'adoentado', 'amargamente', 'antipatia', 'antipático', 'asco', 'asqueroso', 'aversão', 'chatear', 'chateação', 'desagrado', 'desagradável', 'desprezível', 'detestável', 'doente', 'doença', 'enfermidade', 'enjoativo', 'enjoo', 'enjôo', 'feio', 'fétido', 'golfar', 'grave', 'gravidade', 'grosseiro', 'grosso', 'horrível', 'ignóbil', 'ilegal', 'incomodar', 'incômdo', 'indecente', 'indisposição', 'indisposto', 'inescrupuloso', 'maldade', 'maldoso', 'malvado', 'mau', 'nauseabundo', 'nauseante', 'nausear', 'nauseoso', 'nojento', 'nojo', 'náusea', 'obsceno', 'obstruir', 'obstrução', 'ofensivo', 'patético', 'perigoso', 'repelente', 'repelir', 'repugnante', 'repulsa', 'repulsivo', 'repulsão', 'rude', 'sujeira', 'sujo', 'terrivelmente', 'terrível', 'torpe', 'travesso', 'travessura', 'ultrajante', 'vil', 'vomitar', 'vômito',]
MEDO = ['abominável', 'afugentar', 'alarmar', 'alerta', 'ameaça', 'amedrontar', 'angustia', 'angústia', 'angustiadamente', 'ansiedade', 'ansioso', 'apavorar', 'apreender', 'apreensão', 'apreensivo', 'arrepio', 'assombrado', 'assombro', 'assustado', 'assustadoramente', 'atemorizar', 'aterrorizante', 'brutal', 'calafrio', 'chocado', 'chocante', 'consternado', 'covarde', 'cruel', 'crueldade', 'cruelmente', 'cuidado', 'cuidadosamente', 'cuidadoso', 'defender', 'defensor', 'defesa', 'derrotar', 'desconfiado', 'desconfiança', 'desencorajar', 'desespero', 'deter', 'envergonhado', 'escandalizado', 'escuridão', 'espantoso', 'estremecedor', 'estremecer', 'expulsar', 'feio', 'friamente', 'fugir', 'hesitar', 'horrendo', 'horripilante', 'horrível', 'horrivelmente', 'horror', 'horrorizar', 'impaciência', 'impaciente', 'impiedade', 'impiedoso', 'indecisão', 'inquieto', 'insegurança', 'inseguro', 'intimidar', 'medonho', 'medroso', 'monstruosamente', 'mortalha', 'nervoso', 'pânico', 'pavor', 'premonição', 'preocupar', 'presságio', 'pressentimento', 'recear', 'receativamente', 'receio', 'receoso', 'ruim', 'suspeita', 'suspense', 'susto', 'temer', 'tenso', 'terror', 'tremor', 'temeroso', 'terrificar', 'timidamente', 'vigiar', 'temor', 'terrível', 'timidez', 'vigilante', 'tensão', 'terrivelmente', 'tímido',]
RAIVA = ['abominação', 'aborrecer', 'adredido', 'agredir', 'agressão', 'agressivo', 'amaldiçoado', 'amargor', 'amargura', 'amolar', 'angústia', 'animosidade', 'antipatia', 'antipático', 'asco', 'assassinar', 'assassinato', 'assediar', 'assédio', 'atormentar', 'avarento', 'avareza', 'aversão', 'beligerante', 'bravejar', 'chateação', 'chato', 'cobiçoso', 'cólera', 'colérico', 'complicar', 'contraiedade', 'contrariar', 'corrupção', 'corrupto', 'cruxificar', 'demoníaco', 'demônio', 'descaso', 'descontente', 'descontrole', 'desenganar', 'desgostar', 'desgraça', 'desprazer', 'desprezar', 'destruição', 'destruir', 'detestar', 'diabo', 'diabólico', 'doido', 'encolerizar', 'energicamente', 'enfurecido', 'enfuriante', 'enlouquecer', 'enraivecer', 'escandalizar', 'escândalo', 'escoriar', 'exasperar', 'execração', 'ferir', 'frustração', 'frustrar', 'fúria', 'furioso', 'furor', 'ganância', 'ganancioso', 'guerra', 'guerreador', 'guerrilha', 'hostil', 'humilhar', 'implicância', 'implicar', 'importunar', 'incomodar', 'incômodo', 'indignar', 'infernizar', 'inimigo', 'inimizade', 'injúria', 'injuriado', 'injustiça', 'insulto', 'malícia', 'odiável', 'repulsivo', 'inveja', 'malicioso', 'ódio', 'resmungar', 'ira', 'malignidade', 'odioso', 'ressentido', 'irado', 'malígno', 'ofendido', 'revolta', 'irascibilidade', 'maltratar', 'ofensa', 'ridículo', 'irascível', 'maluco', 'opressão', 'tempestuoso', 'irritar', 'malvadeza', 'opressivo', 'tirano', 'louco', 'malvado', 'oprimir', 'tormento', 'loucura', 'matar', 'perseguição', 'torturar', 'magoar', 'mesquinho', 'perseguir', 'ultrage', 'mal', 'misantropia', 'perturbar', 'ultrajar', 'maldade', 'misantrópico', 'perverso', 'vexatório', 'maldição', 'molestar', 'provocar', 'vigoroso', 'maldito', 'moléstia', 'rabugento', 'vingança', 'maldizer', 'mortal', 'raivoso', 'vingar', 'maldoso', 'morte', 'rancor', 'vingativo', 'maleficência', 'mortífero', 'reclamar', 'violência', 'maléfico', 'mortificar', 'repressão', 'violento', 'malevolência', 'nervoso', 'reprimir', 'zangar', 'malévolo', 'odiar', 'repulsa',]
SURPRESA = ['admirar', 'afeição', 'apavorante', 'assombro', 'chocado', 'chocante', 'desconcertar', 'deslumbrar', 'embasbacar', 'emudecer', 'encantamento', 'enorme', 'espanto', 'estupefante', 'estupefato', 'estupefazer', 'expectativa', 'fantasticamente', 'fantástico', 'horripilante', 'imaginário', 'imenso', 'impressionado', 'incrível', 'maravilha', 'milagre', 'mistério', 'misterioso', 'ótimo', 'pasmo', 'perplexo', 'prodígio', 'sensacional', 'surpreendente', 'surpreender', 'suspense', 'susto', 'temor', 'tremendo',]
TRISTEZA = ['abandonar', 'abatido', 'abominável', 'aborrecer', 'abortar', 'afligir', 'aflito', 'aflição', 'agoniar', 'amargo', 'amargor', 'amargura', 'ansiedade', 'arrepender', 'arrependidamente', 'atrito', 'azar', 'cabisbaixo', 'choro', 'choroso', 'chorão', 'coitado', 'compassivo', 'compunção', 'contristador', 'contrito', 'contrição', 'culpa', 'defeituoso', 'degradante', 'deplorável', 'deposição', 'depravado', 'depressivo', 'depressão', 'deprimente', 'deprimir', 'derrota', 'derrubar', 'desalentar', 'desamparo', 'desanimar', 'desapontar', 'desconsolo', 'descontente', 'desculpas', 'desencorajar', 'desespero', 'desgaste', 'desgosto', 'desgraça', 'desistir', 'desistência', 'deslocado', 'desmoralizar', 'desolar', 'desonra', 'despojado', 'desprazer', 'desprezo', 'desumano', 'desânimo', 'discriminar', 'disforia', 'disfórico', 'dissuadir', 'doloroso', 'dor', 'dó', 'enfadado', 'enlutar', 'entediado', 'entristecedor', 'entristecer', 'envergonhar', 'errante', 'erro', 'errôneo', 'escurecer', 'escuridão', 'escuro', 'esquecido', 'estragado', 'execrável', 'extirpar', 'falsidade', 'falso', 'falta', 'fraco', 'fraqueza', 'fricção', 'frieza', 'frio', 'funesto', 'fúnebre', 'grave', 'horror', 'humilhar', 'inconsolável', 'indefeso', 'infelicidade', 'infeliz', 'infortúnio', 'isolar', 'lacrimejante', 'lacrimoso', 'lamentar', 'lastimoso', 'luto', 'lutoso', 'lágrima', 'lástima', 'lúgubre', 'magoar', 'martirizar', 'martírio', 'mau', 'melancolia', 'melancólico', 'menosprezar', 'miseravelmente', 'misterioso', 'mistério', 'miséria', 'morre', 'morte', 'mortificante', 'mágoa', 'negligentemente', 'nocivo', 'obscuro', 'opressivo', 'opressão', 'oprimir', 'pena', 'penalizar', 'penitente', 'penoso', 'penumbra', 'perder', 'perturbado', 'perverso', 'pervertar', 'pesaroso', 'pessimamente', 'piedade', 'pobre', 'porcamente', 'prejudicado', 'prejudicial', 'prejuízo', 'pressionar', 'pressão', 'quebrar', 'queda', 'queixoso', 'rechaçar', 'remorso', 'repressivo', 'repressão', 'reprimir', 'ruim', 'secreto', 'servil', 'sobrecarga', 'sobrecarregado', 'sofrer', 'sofrimento', 'solidão', 'sombrio', 'soturno', 'sujo', 'suplicar', 'suplício', 'só', 'timidez', 'torturar', 'trevas', 'triste', 'tristemente', 'tédio', 'tímido', 'vazio',]

emotion_words = {
    'ALEGRIA': ALEGRIA,
    'DESGOSTO': DESGOSTO,
    'MEDO': MEDO,
    'RAIVA': RAIVA,
    'SURPRESA': SURPRESA,
    'TRISTEZA': TRISTEZA,
}
for key, values in emotion_words.items():
    for i, word in enumerate(values):
        emotion_words[key][i] = ''.join([remover_acentos(p.strip()) for p in LOOKUP.get(word.lower(), word.lower())])
# pp.pprint(emotion_words)

In [33]:
stpwords = stopwords.words('portuguese') + list(punctuation)
rms = ['um', 'não', 'mais', 'muito']
for rm in rms:
    del stpwords[stpwords.index(rm)]

def tokenize_frases(frase):
    return word_tokenize(remover_acentos(frase.lower()))

def rm_stop_words_tokenized(frase):
    frase = nlp(remover_acentos(frase.lower()))
    clean_frase = []
    for palavra in frase:
        if palavra.pos_ != 'PUNCT':
            palavra = palavra.lemma_
            for pc in ['.', ',']:
                palavra = palavra.replace(pc, '')
            if palavra not in stpwords and not palavra.isdigit():
                clean_frase.append(palavra)
    return ' '.join(clean_frase)

def generate_corpus(frases, tokenize=False):
    print('Iniciando processamento...')
    tokenized_frases = frases
    with concurrent.futures.ProcessPoolExecutor(max_workers=4) as procs:
        if tokenize:
            print('Executando processo de tokenização das frases...')
            tokenized_frases = procs.map(tokenize_frases, frases, chunksize=25)
        print('Executando processo de remoção das stopwords...')
        tokenized_frases = procs.map(rm_stop_words_tokenized, tokenized_frases, chunksize=25)
    print('Filtro e finalização...')
    return tokenized_frases


frases_originais = [
    'Bom dia SENADOR, agora está claro porque o pedágio não baixava,o judiciário não se manifestava quando era provocado e as CPIs só serviram prá corrupção,deu no que deu 🙄',
    'Não basta apenas retirar o candidato preferencial da maioria dos eleitores brasileiros. Tem que impedir também que esses mesmos eleitores possam comparecer às urnas. Que democracia é essa, minha gente? Poder judiciário comprometido até os cabelos com o golpe de destrói o país.',
    'Deus abençoe o dia de todos você, tenham um bom trabalho e bom estudo a todos. E pra aqueles que não trabalha e nem estuda, boa curtição em sua cama 🙂',
    'Aprenda a ter amor próprio que nem essa banana q fez uma tatuagem dela mesma.',
    'Estou muito feliz hoje',
    'Dias chuvosos me deixam triste',
    'Hoje o dia esta excelente',
    'Tem certas coisas que eu não como, acho bem nojento ficar mastigando lingua de boi por exemplo',
    'É de se admirar àqueles que conseguem realizar boas ações sem desejar nada em troca',
    
    'Quando a tristeza bater na sua porta, abra um belo sorriso e diga: Desculpa, mas hoje a felicidade chegou primeiro!',
    'Feliz é aquele que vê a felicidade dos outros sem ter inveja. O sol é para todos e a sombra pra quem merece.',
    'Minha meta é ser feliz, não perfeito.',
    'Ser feliz até onde der, até onde puder. Sem adiar, ser feliz o tanto que durar.',
    'Nunca deixe ninguém dizer que você não pode fazer alguma coisa. Se você tem um sonho, tem que correr atrás dele. As pessoas não conseguem vencer, e dizem que você também não vai vencer. Se quer alguma coisa, corre atrás.',
    
    'O maior problema em acreditar nas pessoas erradas, é que um dia você acaba não acreditando em mais ninguém.',
    'Não me deixe ir, posso nunca mais voltar.',
    'Não me arrependo de ter conhecido ninguém, só me arrependo de ter perdido tanto tempo com algumas pessoas.',
    'Se existe uma coisa que eu me arrependo é de ter confiado em algumas pessoas.',
    'Prefiro que enxerguem em mim erros com arrependimento do que uma falsa perfeição.',
    
    'Mesmo sabendo que um dia a vida acaba, a gente nunca está preparado para perder alguém.',
    'A morte é uma pétala que se solta da flor e deixa uma eterna saudade no coração.',
    'Mãe é imortal, porque quando ela parte para outro mundo fica vivendo nas lágrimas que escorrem em nosso rosto eternamente.',
    'Só existem dois motivos pra uma pessoa se preocupar com você: Ou ela te ama muito, ou você tem algo que ela queira muito!',
    'Ser feliz nao é ter uma vida perfeita, mas sim reconhecer que vale a pena viver apesar de todos os desafios e perdas.'
]

# N = 10000

frases = copy.deepcopy(frases_originais)
frases += [' '.join(f).replace('_', ' ') for f in flt.sents()[:1000]]
frases += [' '.join(f).replace('_', ' ') for f in mch.sents()[:1000]]
frases += [' '.join(f).replace('_', ' ') for f in mcm.sents()[:1000]]

frases = list(generate_corpus(frases, tokenize=False))
# print(frases)

ldocs = [f'D{i}' for i in range(len(frases))]

Iniciando processamento...
Executando processo de remoção das stopwords...
Filtro e finalização...


In [34]:
print('Tf-Idf:')
vec_tfidf = TfidfVectorizer(max_df=5, sublinear_tf=False, use_idf=True, ngram_range=(1, 3))
X_tfidf = vec_tfidf.fit_transform(frases)
print("   Actual number of tfidf features: %d" % X_tfidf.get_shape()[1])
weights_tfidf = pd.DataFrame(np.round(X_tfidf.toarray().T, 3), index=vec_tfidf.get_feature_names(), columns=ldocs)
display(weights_tfidf.head(15))

Tf-Idf:
   Actual number of tfidf features: 62098


,D0,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D3014,D3015,D3016,D3017,D3018,D3019,D3020,D3021,D3022,D3023
0h00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0h00 ser,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0h00 ser quatro,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100000 centimetro,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100000 centimetro quadrar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10a,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10a edicao,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10a edicao pregao,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11o,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
print('Count:')
vec_count = CountVectorizer()
X_count = vec_count.fit_transform(frases)
print("   Actual number of tfidf features: %d" % X_count.get_shape()[1])
weights_count = pd.DataFrame(X_count.toarray().T, index=vec_count.get_feature_names(), columns=ldocs)
display(weights_count.head(15))

Count:
   Actual number of tfidf features: 7303


,D0,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D3014,D3015,D3016,D3017,D3018,D3019,D3020,D3021,D3022,D3023
0h00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10a,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11o,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1275,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13a,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14a,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18h30,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1950,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1985,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
USE = True
X = X_count if USE else X_tfidf
weights = weights_count if USE else weights_tfidf
vectorizer = vec_count if USE else vec_tfidf

print('SVD: ')
AC = copy.deepcopy(X.toarray().T)
u, s, v = np.linalg.svd(AC, full_matrices=False)
print('Original and SVD equals: ', np.allclose(AC, np.dot(u, np.dot(np.diag(s), v))))

# print(AC)
# print(u.astype(np.float16))
# print('-' * 20)
# print(np.diag(s.astype(np.float16)))
# print('-' * 20)
# print(v.astype(np.float16))

SVD: 
Original and SVD equals:  True


In [37]:
print('Matriz U:')
print(u.shape)
weights_um = pd.DataFrame(u, index=vectorizer.get_feature_names(), columns=ldocs)
display(weights_um.head(15))

Matriz U:
(7303, 3024)


,D0,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D3014,D3015,D3016,D3017,D3018,D3019,D3020,D3021,D3022,D3023
0h00,-0.000330,0.001034,0.000631,-1.626920e-04,-0.000245,0.000252,0.000243,0.000100,0.000006,0.000072,...,0.001445,-0.006505,0.020359,-0.003658,-0.005072,-0.000573,0.004184,0.013944,0.009463,-0.007827
100000,-0.000103,-0.000043,0.000018,6.426083e-05,0.000151,-0.000191,-0.000834,-0.000032,0.000254,-0.000448,...,0.003179,0.026836,-0.012535,-0.019819,0.014303,-0.001634,-0.025628,-0.009137,-0.022411,-0.014440
10a,-0.000107,-0.000073,0.000071,-5.820491e-05,-0.000007,-0.000184,-0.000659,0.000330,0.000580,-0.000237,...,0.035739,-0.025074,-0.057406,-0.026335,-0.020326,-0.053951,0.005556,-0.020684,0.030122,-0.006779
11o,-0.000593,-0.001574,0.000674,-2.269369e-04,-0.001062,-0.000468,-0.001057,0.002376,0.000025,0.000909,...,-0.010860,0.007196,0.002599,-0.014940,-0.008083,-0.008427,-0.001342,0.006708,-0.017152,-0.010789
1275,-0.000339,0.001014,0.000614,-1.900729e-04,-0.000207,0.000278,0.000220,0.000083,0.000049,0.000060,...,-0.010549,-0.013339,0.055930,-0.006550,-0.010075,0.009329,0.020917,0.036038,0.003070,0.019489
13a,-0.000655,0.000680,0.000788,2.040629e-05,-0.000059,-0.000397,-0.002271,0.001430,-0.000753,-0.003935,...,-0.007292,0.023522,-0.010375,0.009805,-0.002094,-0.011149,-0.027253,-0.009715,0.003255,0.000944
14a,-0.000432,0.000886,0.000747,9.627090e-07,-0.000265,-0.000301,-0.001130,0.001254,0.000190,-0.000245,...,-0.004928,0.024048,-0.019266,-0.006951,0.021694,-0.010548,-0.023005,-0.001699,-0.021148,-0.005433
18h30,-0.001119,-0.000584,0.001048,-2.945792e-04,0.000775,0.001604,0.001108,-0.000021,0.000144,-0.000056,...,0.025003,0.003549,-0.010260,-0.025247,0.016008,0.005207,-0.017907,0.013774,-0.016031,0.018314
1950,-0.000799,-0.001753,0.000714,-1.254287e-04,0.000827,0.000839,-0.000464,0.001603,-0.000041,0.001082,...,0.031058,0.020538,0.008320,-0.000015,-0.026407,-0.066242,0.025046,0.020347,0.001191,0.041966
1985,-0.000799,-0.001753,0.000714,-1.254287e-04,0.000827,0.000839,-0.000464,0.001603,-0.000041,0.001082,...,-0.018112,0.051022,0.058081,0.015958,-0.016342,-0.071696,0.001718,0.027118,-0.030544,-0.004389


In [38]:
print('Matriz VT:')
weights_vm = pd.DataFrame(v.T, columns=ldocs)
display(weights_vm.head(15))

Matriz VT:


,D0,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D3014,D3015,D3016,D3017,D3018,D3019,D3020,D3021,D3022,D3023
0,-0.036009,0.034462,-0.055872,0.017273,-0.020700,0.035030,-0.000187,0.016615,-0.005359,0.010104,...,-0.000000e+00,-0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-0.000000e+00,0.000000e+00,0.000000e+00
1,-0.020624,0.006020,-0.045104,0.017253,-0.005518,-0.027266,0.019154,0.003210,-0.006928,-0.005015,...,-1.520283e-18,-1.854470e-17,-2.510833e-17,5.555589e-18,-5.864454e-18,-4.062573e-18,-1.242058e-17,5.263500e-18,-2.865046e-17,4.954048e-17
2,-0.033057,-0.016772,-0.045427,-0.016481,-0.016604,-0.034353,0.031752,0.000316,0.007801,-0.000575,...,1.301072e-17,4.052942e-18,-9.587507e-17,2.668051e-17,-1.693507e-17,-1.163555e-17,3.142488e-17,-6.418208e-17,-3.747925e-17,3.266560e-17
3,-0.013727,-0.004746,-0.009331,0.000727,0.046035,-0.031503,0.031868,-0.000003,0.004511,0.003505,...,9.069362e-17,2.251190e-17,-1.228769e-17,-5.660848e-17,-1.374043e-16,-7.275480e-17,-2.279700e-17,-3.012215e-17,9.774903e-17,-3.386826e-17
4,-0.003289,0.000379,-0.001047,0.002061,0.001410,-0.005187,-0.007495,-0.001877,0.004214,-0.008846,...,-4.432477e-16,4.205333e-16,8.320222e-18,8.748729e-17,3.319394e-16,5.492552e-17,1.138556e-16,2.821030e-16,1.912016e-16,-1.667124e-16
5,-0.001977,-0.001554,-0.000825,-0.001061,0.000060,-0.000261,-0.005070,0.000702,0.007353,0.000195,...,1.186928e-16,5.405597e-17,4.416293e-17,-3.464185e-17,-7.522711e-18,4.601636e-17,8.503870e-17,-1.836351e-17,-1.061303e-16,-1.111751e-16
6,-0.002048,-0.002085,0.000381,-0.001081,0.000252,-0.001485,-0.005327,0.002264,0.008043,-0.000966,...,2.311124e-16,-2.090790e-16,1.990240e-16,-1.059818e-17,-2.580552e-17,-3.384599e-17,4.279696e-17,5.081040e-17,2.141100e-16,1.488553e-16
7,-0.018075,0.003781,-0.043803,0.013668,-0.003774,-0.027079,0.021097,-0.005594,-0.026719,-0.047739,...,8.727657e-17,-3.588692e-17,3.277007e-17,-5.832197e-17,-1.486871e-17,-1.232932e-16,-6.342464e-17,8.777961e-17,-8.714659e-17,-1.221196e-16
8,-0.002387,0.000065,-0.001705,0.000715,0.000877,-0.001420,-0.001491,0.000061,0.006047,-0.002699,...,-1.632344e-17,-9.461889e-17,-9.586003e-17,-1.227433e-16,1.297191e-16,-9.627393e-17,2.394475e-16,6.264316e-17,-6.354243e-18,1.156811e-16
9,-0.017179,-0.018555,-0.005412,-0.028427,-0.011422,-0.011516,-0.022334,-0.010742,0.048277,-0.006552,...,5.374827e-17,5.525176e-18,-1.768747e-17,-7.927074e-17,-1.372445e-17,8.012188e-18,1.195985e-17,6.323412e-17,-3.498241e-18,3.755969e-17


In [39]:
SIMPLE = USE

ws = np.zeros((len(ldocs), len(emotion_words.keys())))
idx = { w:i for i, w in enumerate(weights.index.get_values())}

for i, doc in enumerate(ldocs):
    for k, item in enumerate(emotion_words.items()):
        key, values = item
        for value in values:
            try:
                if SIMPLE:
                    index = weights.index.get_loc(value)
                    idx_val = u[index]
                    ws[i][k] += idx_val[i] * weights.iloc[index].values[i]
                else:
                    weight_sum = []
                    indexes = filter(None, [e if value in inx else None for e, inx in enumerate(idx.keys())])
                    for index in indexes:
                        idx_val = u[index]
                        weight_sum.append(idx_val[i] * weights.iloc[index].values[i])
                    ws[i][k] += np.sum(weight_sum)
            except:
                pass

ws = ws/len(ldocs)
            
df = pd.DataFrame(ws, columns=emotion_words.keys())
display(df[
    (df['ALEGRIA'] != 0) | (df['DESGOSTO'] != 0) | (df['MEDO'] != 0) | 
    (df['RAIVA'] != 0) | (df['SURPRESA'] != 0) | (df['TRISTEZA'] != 0)
])

,ALEGRIA,DESGOSTO,MEDO,RAIVA,SURPRESA,TRISTEZA
0,-1.107802e-05,0.000000e+00,0.000000,-2.765431e-06,0.000000,-1.085174e-05
2,-1.452364e-05,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00
3,-4.719741e-06,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00
4,9.382981e-07,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00
5,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,5.444077e-07
6,9.562855e-07,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00
7,0.000000e+00,-1.088535e-07,0.000000,0.000000e+00,0.000000,0.000000e+00
8,1.960939e-05,0.000000e+00,0.000000,0.000000e+00,0.000002,0.000000e+00
9,-3.224654e-07,0.000000e+00,0.000000,0.000000e+00,0.000000,-1.866379e-06
10,-1.856638e-06,0.000000e+00,0.000000,-1.112136e-07,0.000000,0.000000e+00


In [49]:
dtframe = np.zeros((len(ldocs), len(emotion_words.keys())))
for i, d in enumerate(ldocs):
    for k, it in enumerate(emotion_words.items()):
        a = [ws[:, k]]
        b = [v.T[i]]
        dtframe[i][k] = cosine_similarity(a, b)
dtframe = np.round(normalization(dtframe, -100, 100), 2)
       
for i, frase in enumerate(frases_originais[9:24]):
    print(f' D{i+9} - {frase}')

df = pd.DataFrame(dtframe[9:24].T, index=emotion_words.keys(), columns=ldocs[9:24])
display(df.head(15))

 D9 - Quando a tristeza bater na sua porta, abra um belo sorriso e diga: Desculpa, mas hoje a felicidade chegou primeiro!
 D10 - Feliz é aquele que vê a felicidade dos outros sem ter inveja. O sol é para todos e a sombra pra quem merece.
 D11 - Minha meta é ser feliz, não perfeito.
 D12 - Ser feliz até onde der, até onde puder. Sem adiar, ser feliz o tanto que durar.
 D13 - Nunca deixe ninguém dizer que você não pode fazer alguma coisa. Se você tem um sonho, tem que correr atrás dele. As pessoas não conseguem vencer, e dizem que você também não vai vencer. Se quer alguma coisa, corre atrás.
 D14 - O maior problema em acreditar nas pessoas erradas, é que um dia você acaba não acreditando em mais ninguém.
 D15 - Não me deixe ir, posso nunca mais voltar.
 D16 - Não me arrependo de ter conhecido ninguém, só me arrependo de ter perdido tanto tempo com algumas pessoas.
 D17 - Se existe uma coisa que eu me arrependo é de ter confiado em algumas pessoas.
 D18 - Prefiro que enxerguem em mim err

,D9,D10,D11,D12,D13,D14,D15,D16,D17,D18,D19,D20,D21,D22,D23
ALEGRIA,-22.26,-19.63,-35.11,-42.87,22.42,-1.90,-8.35,-20.56,-20.89,3.80,-39.12,-6.13,-50.54,-34.14,2.92
DESGOSTO,-8.88,-32.90,-21.35,-26.85,-18.58,-3.17,22.58,-15.08,-19.13,-12.74,-46.23,-11.83,1.63,-4.89,-2.24
MEDO,-16.23,-28.50,-18.58,-2.33,-37.86,1.01,-19.32,-7.63,-12.95,-11.59,-0.50,8.26,-21.58,-5.88,-55.16
RAIVA,-16.76,-21.91,1.41,-23.34,-45.74,4.55,-46.04,-60.56,-3.54,-19.55,-16.84,-47.10,33.12,-41.20,-5.70
SURPRESA,1.37,-26.86,3.97,-10.58,-23.18,-39.04,-11.25,-23.85,-35.70,-31.62,-49.01,-17.10,-14.48,-7.57,-0.41
TRISTEZA,-4.91,-25.33,-9.44,-17.74,9.28,7.93,-4.98,-14.31,-5.01,-14.52,-32.87,-6.84,-4.00,14.14,-34.58
